In [194]:
# import the required libraries
import os
import re
import csv
import pandas as pd
import numpy as np

In [195]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_file = os.path.join(folderlink+input_folder+'test_data.csv')
input_pred = folderlink+input_folder+"new_predikanten.csv"


In [196]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [197]:
jaar_vertrek_type = {'jaar vertrek': pd.Int64Dtype(), 'ind_id': pd.Int64Dtype(),'dag intrede': pd.Int64Dtype(), 'dag vertrek': pd.Int64Dtype() }
df = pd.read_csv(input_pred, sep=',', dtype= jaar_vertrek_type, encoding='utf-8')

In [198]:
df.head(10)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden
0,35651,1,ZH,NaN,Oudewater,NaN,(Hekendorp) Brunner; L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN
1,45546,2,GE,NaN,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,NaN
2,7179,3,NB,NaN,Breda,NaN,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
3,21505,3,GR,NaN,Hornhuizen,NaN,Aa; dr. P.J.B.K. Simon van der,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN
4,36887,3,NH,NaN,Purmerend,NaN,Aa; dr. P.J.B.K. Simon van der,Goutum,<NA>,NaN,1886,Winterswijk,<NA>,NaN,1892,NaN
5,50146,3,GE,NaN,Winterswijk,NaN,Aa; dr. P.J.B.K. Simon van der,Purmerend,<NA>,NaN,1892,Breda,<NA>,NaN,1893,NaN
6,15179,3,FR,NaN,Goutum,NaN,Aa; dr. P.J.B.K. van der,Kloosterburen,<NA>,NaN,1866,Purmerend,<NA>,NaN,1886,NaN
7,7204,3,NB,NaN,Breda,4. Wijk Noord,Aa; dr. P.J.B.K.S. van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
8,19779,4,GE,NaN,Herwijnen,NaN,Aa; W.P. van der,kandidaat,24,december,1989,NaN,<NA>,NaN,<NA>,NaN
9,35941,5,OV,NaN,Overdinkel,NaN,Aaij; J.,Hoogkarspel-De Drieslag,25,september,1983,emeritaat,1,september,1986,NaN


In [199]:
# The dataset is split in two. Since the dataset is composed of rows that have information about where someone went to ('vertrek naar of vanwege') and where he or she came from ('Herkomst'), which is foremost for ministers from after 1816, and rows that have only a year when they move position a cut was made based on the field Herkomst having values NaN.
# The former is named dm_part2, the latter dm_part1. 

dm_part1 = df[df['Herkomst'].isna()]

In [200]:
dm_part1.head(10)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden
36,35685,<NA>,NH,Alkmaar,Oudkarspel,NaN,Aalburg; Johannes van,NaN,<NA>,NaN,1743,NaN,<NA>,NaN,1777,NaN
62,22489,<NA>,NH,Woerden,Kalslagen,NaN,Aalst; Cornelius van,NaN,<NA>,NaN,1717,NaN,<NA>,NaN,1751,NaN
68,46953,<NA>,GE,Zaltbommel,Vuren,NaN,Aalst; Gerardus van,NaN,<NA>,NaN,1704,NaN,<NA>,NaN,1706,NaN
69,41854,<NA>,ZH,Schouwen-Duiveland,Sommelsdijk,NaN,Aalst; Gerardus van,NaN,<NA>,NaN,1706,NaN,<NA>,NaN,1715,NaN
70,48641,<NA>,NH,Haarlem,West-Zaandam,NaN,Aalst; Gerardus van,NaN,<NA>,NaN,1715,NaN,<NA>,NaN,1755,NaN
71,741,<NA>,ZE,Walcheren,Aardenburg,NaN,Aalst; Wilhelmus,NaN,<NA>,NaN,1695,NaN,<NA>,NaN,1700,NaN
72,8276,<NA>,NH,Haarlem,Castricum,NaN,Aalstius; Henricus,NaN,<NA>,NaN,1700,NaN,<NA>,NaN,1733,NaN
73,32102,<NA>,NH,Hoorn,Obdam,NaN,Aalstius; Johannes,NaN,<NA>,NaN,1644,NaN,<NA>,NaN,1648,NaN
74,6933,<NA>,NB,’s-Hertogenbosch,Boxtel,NaN,Aalstius; Johannes,NaN,<NA>,NaN,1648,NaN,<NA>,NaN,1652,NaN
75,32398,<NA>,NB,Peel- en Kempenland,Oirschot,NaN,Aalstius; Johannes,NaN,<NA>,NaN,1648,NaN,<NA>,NaN,1657,NaN


In [201]:
dm_part1['predikant'] = df['predikant'].str.replace(' ', '')
dm_part1['pred_year_start'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar intrede'].astype(str)
dm_part1['pred_year_end'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar vertrek'].astype(str)

C:\Windows\Temp\ipykernel_3084\1813838777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part1['predikant'] = df['predikant'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_3084\1813838777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part1['pred_year_start'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar intrede'].astype(str)
C:\Windows\Temp\ipykernel_3084\1813838777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [202]:
tdf = dm_part1
variable_end = 'pred_year_end'
variable_start = 'pred_year_start'
variable_name = 'predikant'

In [203]:
def assign_ids(tdf):
    individuals = {}
    id_counter = 1

    def get_individual_id(name, start, end):
        if name in individuals:
            if individuals[name][variable_end] == start:
                individuals[name][variable_end] = end
                return individuals[name]['ID']
            else:
                nonlocal id_counter
                id_counter += 1
                individuals[name] = {'ID': id_counter, variable_start: start, variable_end: end}
                return id_counter
        else:
            individuals[name] = {'ID': id_counter, variable_start: start, variable_end: end}
            return id_counter

    tdf['ID'] = tdf.apply(lambda row: get_individual_id(row[variable_name], row[variable_start], row[variable_end]), axis=1)
    return tdf

result_df = assign_ids(tdf)

C:\Windows\Temp\ipykernel_3084\1319283031.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['ID'] = tdf.apply(lambda row: get_individual_id(row[variable_name], row[variable_start], row[variable_end]), axis=1)


In [204]:
result_dmpart1 = result_df
result_dmpart1.head(100)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,pred_year_start,pred_year_end,ID
36,35685,<NA>,NH,Alkmaar,Oudkarspel,NaN,Aalburg;Johannesvan,NaN,<NA>,NaN,1743,NaN,<NA>,NaN,1777,NaN,Aalburg;Johannesvan_1743,Aalburg;Johannesvan_1777,1
62,22489,<NA>,NH,Woerden,Kalslagen,NaN,Aalst;Corneliusvan,NaN,<NA>,NaN,1717,NaN,<NA>,NaN,1751,NaN,Aalst;Corneliusvan_1717,Aalst;Corneliusvan_1751,1
68,46953,<NA>,GE,Zaltbommel,Vuren,NaN,Aalst;Gerardusvan,NaN,<NA>,NaN,1704,NaN,<NA>,NaN,1706,NaN,Aalst;Gerardusvan_1704,Aalst;Gerardusvan_1706,1
69,41854,<NA>,ZH,Schouwen-Duiveland,Sommelsdijk,NaN,Aalst;Gerardusvan,NaN,<NA>,NaN,1706,NaN,<NA>,NaN,1715,NaN,Aalst;Gerardusvan_1706,Aalst;Gerardusvan_1715,1
70,48641,<NA>,NH,Haarlem,West-Zaandam,NaN,Aalst;Gerardusvan,NaN,<NA>,NaN,1715,NaN,<NA>,NaN,1755,NaN,Aalst;Gerardusvan_1715,Aalst;Gerardusvan_1755,1
71,741,<NA>,ZE,Walcheren,Aardenburg,NaN,Aalst;Wilhelmus,NaN,<NA>,NaN,1695,NaN,<NA>,NaN,1700,NaN,Aalst;Wilhelmus_1695,Aalst;Wilhelmus_1700,1
72,8276,<NA>,NH,Haarlem,Castricum,NaN,Aalstius;Henricus,NaN,<NA>,NaN,1700,NaN,<NA>,NaN,1733,NaN,Aalstius;Henricus_1700,Aalstius;Henricus_1733,1
73,32102,<NA>,NH,Hoorn,Obdam,NaN,Aalstius;Johannes,NaN,<NA>,NaN,1644,NaN,<NA>,NaN,1648,NaN,Aalstius;Johannes_1644,Aalstius;Johannes_1648,1
74,6933,<NA>,NB,’s-Hertogenbosch,Boxtel,NaN,Aalstius;Johannes,NaN,<NA>,NaN,1648,NaN,<NA>,NaN,1652,NaN,Aalstius;Johannes_1648,Aalstius;Johannes_1652,1
75,32398,<NA>,NB,Peel- en Kempenland,Oirschot,NaN,Aalstius;Johannes,NaN,<NA>,NaN,1648,NaN,<NA>,NaN,1657,NaN,Aalstius;Johannes_1648,Aalstius;Johannes_1657,2


In [205]:
result_dmpart1['individual'] = result_dmpart1[variable_name].astype(str)+'_'+result_dmpart1['ID'].astype(str) 

C:\Windows\Temp\ipykernel_3084\886171708.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_dmpart1['individual'] = result_dmpart1[variable_name].astype(str)+'_'+result_dmpart1['ID'].astype(str)


In [206]:
unique_individuals_table_p1 = result_dmpart1[[variable_name,'individual','ID']].copy()

In [207]:
unique_individuals_table_p1 = unique_individuals_table_p1.drop_duplicates(subset=['individual'])

In [208]:
unique_individuals_table_p1.describe()

,ID
count,14240.000000
mean,1009.726545
std,566.461110
min,1.000000
25%,528.000000
50%,1010.000000
75%,1512.000000
max,1972.000000


In [209]:
unique_individuals_table_p1.head(50)

,predikant,individual,ID
36,Aalburg;Johannesvan,Aalburg;Johannesvan_1,1
62,Aalst;Corneliusvan,Aalst;Corneliusvan_1,1
68,Aalst;Gerardusvan,Aalst;Gerardusvan_1,1
71,Aalst;Wilhelmus,Aalst;Wilhelmus_1,1
72,Aalstius;Henricus,Aalstius;Henricus_1,1
73,Aalstius;Johannes,Aalstius;Johannes_1,1
75,Aalstius;Johannes,Aalstius;Johannes_2,2
76,Aalstius;Johannes,Aalstius;Johannes_3,3
77,Aalstius;Johannes,Aalstius;Johannes_4,4
80,Aalstius;Leonardus,Aalstius;Leonardus_4,4


In [210]:
dm_part2 = df[df['Herkomst'].notna()]

In [211]:
dm_part2.head(50)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden
0,35651,1,ZH,NaN,Oudewater,NaN,(Hekendorp) Brunner; L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN
1,45546,2,GE,NaN,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,NaN
2,7179,3,NB,NaN,Breda,NaN,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
3,21505,3,GR,NaN,Hornhuizen,NaN,Aa; dr. P.J.B.K. Simon van der,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN
4,36887,3,NH,NaN,Purmerend,NaN,Aa; dr. P.J.B.K. Simon van der,Goutum,<NA>,NaN,1886,Winterswijk,<NA>,NaN,1892,NaN
5,50146,3,GE,NaN,Winterswijk,NaN,Aa; dr. P.J.B.K. Simon van der,Purmerend,<NA>,NaN,1892,Breda,<NA>,NaN,1893,NaN
6,15179,3,FR,NaN,Goutum,NaN,Aa; dr. P.J.B.K. van der,Kloosterburen,<NA>,NaN,1866,Purmerend,<NA>,NaN,1886,NaN
7,7204,3,NB,NaN,Breda,4. Wijk Noord,Aa; dr. P.J.B.K.S. van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
8,19779,4,GE,NaN,Herwijnen,NaN,Aa; W.P. van der,kandidaat,24,december,1989,NaN,<NA>,NaN,<NA>,NaN
9,35941,5,OV,NaN,Overdinkel,NaN,Aaij; J.,Hoogkarspel-De Drieslag,25,september,1983,emeritaat,1,september,1986,NaN


In [212]:
dm_part2['gemeente'] = dm_part2['gemeente'].str.replace(' ', '')
dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str.replace(' ', '')

dm_part2['gemeente'] = dm_part2['gemeente'].str[:6]
dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str[:6]


C:\Windows\Temp\ipykernel_3084\541579337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['gemeente'] = dm_part2['gemeente'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_3084\541579337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_3084\541579337.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [213]:
dm_part2['predikant'] = dm_part2['predikant'].str.replace(' ', '')
dm_part2['pred_start'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['gemeente'].astype(str)+'_'+dm_part2['jaar intrede'].astype(str)
dm_part2['pred_end'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['vertrek naar of vanwege'].astype(str)+'_'+dm_part2['jaar vertrek'].astype(str)

C:\Windows\Temp\ipykernel_3084\4125051709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['predikant'] = dm_part2['predikant'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_3084\4125051709.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['pred_start'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['gemeente'].astype(str)+'_'+dm_part2['jaar intrede'].astype(str)
C:\Windows\Temp\ipykernel_3084\4125051709.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [214]:
dm_part2 = dm_part2.sort_values(by='pred_start')

In [215]:
tdf = dm_part2
variable_end = 'pred_end'
variable_start = 'pred_start'
variable_name = 'predikant'


In [216]:
def assign_ids(tdf):
    individuals = {}
    id_counter = 1

    def get_individual_id(name, start, end):
        if name in individuals:
            if individuals[name][variable_end] == start:
                individuals[name][variable_end] = end
                return individuals[name]['ID']
            else:
                nonlocal id_counter
                id_counter += 1
                individuals[name] = {'ID': id_counter, variable_start: start, variable_end: end}
                return id_counter
        else:
            individuals[name] = {'ID': id_counter, variable_start: start, variable_end: end}
            return id_counter

    tdf['ID'] = tdf.apply(lambda row: get_individual_id(row[variable_name], row[variable_start], row[variable_end]), axis=1)
    return tdf

result_df = assign_ids(tdf)

In [217]:
result_dmpart2 = result_df

In [218]:
result_dmpart2['individual'] = result_dmpart2[variable_name].astype(str)+'_'+result_dmpart2['ID'].astype(str) 

In [219]:
unique_individuals_table_p2 = result_dmpart2[[variable_name,'individual','ID']].copy()

In [220]:
unique_individuals_table_p2 = unique_individuals_table_p2.drop_duplicates(subset=['individual'])

In [221]:
unique_individuals_table_p2.describe()

,ID
count,26775.000000
mean,7020.833763
std,4024.837131
min,1.000000
25%,3543.000000
50%,7098.000000
75%,10504.000000
max,13946.000000


In [222]:
unique_individuals_table_p2.head(50)

,predikant,individual,ID
45276,'tHofop;W.J.,'tHofop;W.J._1,1
0,(Hekendorp)Brunner;L.,(Hekendorp)Brunner;L._1,1
1,(Hiltenvan-)Matthijsen;mevrouwL.,(Hiltenvan-)Matthijsen;mevrouwL._1,1
8,Aa;W.P.vander,Aa;W.P.vander_1,1
7,Aa;dr.P.J.B.K.S.vander,Aa;dr.P.J.B.K.S.vander_1,1
3,Aa;dr.P.J.B.K.Simonvander,Aa;dr.P.J.B.K.Simonvander_1,1
4,Aa;dr.P.J.B.K.Simonvander,Aa;dr.P.J.B.K.Simonvander_2,2
6,Aa;dr.P.J.B.K.vander,Aa;dr.P.J.B.K.vander_2,2
2,Aa;drPieterJanBaptistKarelSimonvander,Aa;drPieterJanBaptistKarelSimonvander_2,2
9,Aaij;J.,Aaij;J._2,2


In [223]:
result_dmpart2.head(50)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,pred_start,pred_end,ID,individual
45276,18099,<NA>,GE,NaN,Hedel,NaN,'tHofop;W.J.,kandidaat,1,januari,1973,Ouddor,20,december,1978,NaN,'tHofop;W.J._Hedel_1973,'tHofop;W.J._Ouddor_1978,1,'tHofop;W.J._1
0,35651,1,ZH,NaN,Oudewa,NaN,(Hekendorp)Brunner;L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN,(Hekendorp)Brunner;L._Oudewa_1867,(Hekendorp)Brunner;L._Lobith_1868,1,(Hekendorp)Brunner;L._1
1,45546,2,GE,NaN,Velp,2. Oude Jan,(Hiltenvan-)Matthijsen;mevrouwL.,kandidaat,2,maart,1986,Apeldo,23,oktober,1994,NaN,(Hiltenvan-)Matthijsen;mevrouwL._Velp_1986,(Hiltenvan-)Matthijsen;mevrouwL._Apeldo_1994,1,(Hiltenvan-)Matthijsen;mevrouwL._1
8,19779,4,GE,NaN,Herwij,NaN,Aa;W.P.vander,kandidaat,24,december,1989,NaN,<NA>,NaN,<NA>,NaN,Aa;W.P.vander_Herwij_1989,Aa;W.P.vander_nan_<NA>,1,Aa;W.P.vander_1
7,7204,3,NB,NaN,Breda,4. Wijk Noord,Aa;dr.P.J.B.K.S.vander,Winterswijk,26,februari,1893,emerit,1,mei,1904,NaN,Aa;dr.P.J.B.K.S.vander_Breda_1893,Aa;dr.P.J.B.K.S.vander_emerit_1904,1,Aa;dr.P.J.B.K.S.vander_1
3,21505,3,GR,NaN,Hornhu,NaN,Aa;dr.P.J.B.K.Simonvander,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN,Aa;dr.P.J.B.K.Simonvander_Hornhu_1862,Aa;dr.P.J.B.K.Simonvander_Goutum_1866,1,Aa;dr.P.J.B.K.Simonvander_1
4,36887,3,NH,NaN,Purmer,NaN,Aa;dr.P.J.B.K.Simonvander,Goutum,<NA>,NaN,1886,Winter,<NA>,NaN,1892,NaN,Aa;dr.P.J.B.K.Simonvander_Purmer_1886,Aa;dr.P.J.B.K.Simonvander_Winter_1892,2,Aa;dr.P.J.B.K.Simonvander_2
5,50146,3,GE,NaN,Winter,NaN,Aa;dr.P.J.B.K.Simonvander,Purmerend,<NA>,NaN,1892,Breda,<NA>,NaN,1893,NaN,Aa;dr.P.J.B.K.Simonvander_Winter_1892,Aa;dr.P.J.B.K.Simonvander_Breda_1893,2,Aa;dr.P.J.B.K.Simonvander_2
6,15179,3,FR,NaN,Goutum,NaN,Aa;dr.P.J.B.K.vander,Kloosterburen,<NA>,NaN,1866,Purmer,<NA>,NaN,1886,NaN,Aa;dr.P.J.B.K.vander_Goutum_1866,Aa;dr.P.J.B.K.vander_Purmer_1886,2,Aa;dr.P.J.B.K.vander_2
2,7179,3,NB,NaN,Breda,NaN,Aa;drPieterJanBaptistKarelSimonvander,Winterswijk,26,februari,1893,emerit,1,mei,1904,NaN,Aa;drPieterJanBaptistKarelSimonvander_Breda_1893,Aa;drPieterJanBaptistKarelSimonvander_emerit_1904,2,Aa;drPieterJanBaptistKarelSimonvander_2
